In [2]:
# import FunctionalCollections
import Iterators
import Pipe
import Compat
import JLD
@everywhere using FunctionalCollections
@everywhere using Iterators
@everywhere using Pipe
@everywhere using Compat
@everywhere using JLD

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       

In [3]:
@everywhere fold_indexes=load("brown_glove_folds.jld","fold_indexes")

@everywhere function fold_split(fold_ii, raw_bow_res)
    ground_sents = Vector{ASCIIString}[rset[1] for rset in raw_bow_res]
    reconstructed_bows = Vector{ASCIIString}[rset[2] for rset in raw_bow_res]
    
    test_indexes = fold_indexes[fold_ii]
    training_indexes = trues(ground_sents)
    training_indexes[test_indexes]=false

    test_unordered_sents = reconstructed_bows[test_indexes]
    test_ground = ground_sents[test_indexes]
    training_sents = ground_sents[training_indexes]
    test_unordered_sents,test_ground, training_sents
end
    

In [4]:
@everywhere typealias S ASCIIString
@everywhere typealias State{T} Tuple{T,T}

import PyCall
@everywhere using PyCall
#http://www.nltk.org/howto/probability.html
@everywhere @pyimport nltk
@everywhere @pyimport nltk.probability as nltk_prob
@everywhere const START_MARKER1 = "**START1**"
@everywhere const START_MARKER2 = "**START2**"
@everywhere const END_MARKER1 = "**END1**"
@everywhere const END_MARKER2 = "**END2**"

@everywhere function train_language_model{T}(train_corpus::Vector{Vector{T}}; loglikelyhood::Bool=false)
    function py_collect(xs::PyObject)
        xst = []
        for x in xs
            push!(xst,x)
        end
        xst
    end
    function trigram_buffer(sent)
        [START_MARKER1, START_MARKER2, sent..., END_MARKER1, END_MARKER2] 
    end

    training_trigrams = vcat([py_collect(nltk.trigrams(trigram_buffer(sent))) for sent in train_corpus]...)
    kn_prob_dist = nltk_prob.KneserNeyProbDist(pycall(nltk_prob.FreqDist, PyObject, training_trigrams))
    
    function trigram_model(given1::S, given2::S, event::S)
        prob = kn_prob_dist[:prob]((given1, given2, event))
        loglikelyhood ? log(prob) : prob
    end
end

/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: UserWarning: nltk.draw package not loaded (please install Tkinter library).
  warnings.warn("nltk.draw package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: UserWarning: nltk.draw package not loaded (please install Tkinter library).
  warnings.warn("nltk.draw package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: 

In [5]:
@pyimport nltk.corpus as nltk_corpus
corpus_reader=nltk_corpus.brown
corpus = Vector{ASCIIString}[[lowercase(word) for word in sent] for sent in (corpus_reader[:sents]()|> collect)]
const log_lm = train_language_model(corpus, loglikelyhood=true)

trigram_model (generic function with 1 method)

In [118]:



"""
returns the a vector of sets of node indexes, each set is a subtour
The First subtour returned is the nonconnected one -- the path
"""
function get_subtours(x::Matrix{JuMP.Variable})
    x_val = getValue(x)
    x_iis,x_jjs, _  = findnz(x_val .>= 1 - 1e-6) #It just has to be close to 1 to be true
    nodes_chain = Dict([ii=>jj for (ii,jj) in zip(x_iis,x_jjs)])

    subtours = IntSet[]
    ii = START_NODE_INDEX
    push!(subtours, IntSet(END_NODE_INDEX)) #The END Node is always in the same subtour as the start node
    while(true)
        while(true) #Cycle through current subtour
            
            push!(subtours[end],ii) 
            
            jj = nodes_chain[ii]
            println(ii," ", jj)
            delete!(nodes_chain,ii)
            if jj∈subtours[end] 
                break 
            end
            ii=jj
        end   

        if length(nodes_chain)>0
            ii = first(keys(nodes_chain)) #start new subtour
            push!(subtours,IntSet())
        else
            break
        end
    end
    subtours
end

function get_hyperclass(subtour)
    tour_hyper_class_nodes=IntSet()
    for ii in subtour
        w1,w2 = nodes[ii]
        #println(unordered_markers[w1])
        class_nodes = node_indexes_for_1st[w1]
        union!(tour_hyper_class_nodes,class_nodes) 
    end
    tour_hyper_class_nodes
end


get_hyperclass (generic function with 1 method)

In [122]:

test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","first","performed","in","1892", "."])
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","."])
#
#test_bag =  shuffle(["this","is","the" ,"basis","of","a","fine","comedy", "."])
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy", "."])
#test_bag =  shuffle(["it", "is", "so", "very", "good", "."])
#test_bag =  shuffle(["it", "is", "very", "good", "."])
#test_bag =  shuffle(["it", "is", "good", "."])
#test_bag =  ["no", "way", "."]
#test_bag =  shuffle(["no", "."])
#@time best_order(test_bag, lm, mem_limit=1000)

14-element Array{ASCIIString,1}:
 "in"       
 "1892"     
 "performed"
 "of"       
 "this"     
 "."        
 "the"      
 "first"    
 "of"       
 "manners"  
 "is"       
 "basis"    
 "comedy"   
 "a"        

In [123]:
using JuMP
using MathProgBase
using GLPKMathProgInterface
tic()
m=Model(solver=GLPKSolverMIP())#BonminNLSolver())
number_of_constraints=0


unordered_words  = test_bag
unordered_markers = [START_MARKER1; START_MARKER2; END_MARKER1; END_MARKER2; unordered_words...]
#Note that this lacks END_MARKER2
const START_NODE_INDEX = 1
const END_NODE_INDEX = 2

nodes = State{Int}[] #Named by word index

node_indexes_for_1st = Dict{Int, Vector{Int}}()
node_indexes_for_2nd = Dict{Int, Vector{Int}}()

function add_node!(ii,jj)
    push!(nodes, (ii,jj))

    node_indexes_for_i_1st = get!(()->Int[], node_indexes_for_1st, ii)
    push!(node_indexes_for_i_1st, length(nodes))

    node_indexes_for_j_2nd = get!(()->Int[], node_indexes_for_2nd, jj) 
    push!(node_indexes_for_j_2nd, length(nodes))
    #println("node:$(length(nodes)) |  $(unordered_markers[ii])($ii), $(unordered_markers[jj])($jj)")
end

add_node!(1, 2) #That is START_MARKER1-> START_MARKER2
add_node!(3, 4) #That is END_MARKER1-> END_MARKER2

for ii in 1:length(unordered_markers)
    wi = unordered_markers[ii]
    if wi∈(END_MARKER1,START_MARKER1) continue end  #Covered these
        
    for jj in [1:length(unordered_markers)]
        if ii==jj continue end
        wj = unordered_markers[jj]
        if wj∉(START_MARKER1,START_MARKER2,END_MARKER2)
            #but wj can be  END_MARKER1
            add_node!(ii,jj)
        end
    end
end

@defVar(m, x[1:length(nodes), 1:length(nodes)], Bin)

#If you enter a node you must also leave it
for (cc, center_node) in enumerate(nodes)
    if cc ∉ (START_NODE_INDEX, END_NODE_INDEX) #the beginning and end done have this requiement
        #Everything that enters this node, must leave this node
        @addConstraint(m, sum{x[ii,cc], ii=1:length(nodes)} == sum{x[cc,jj], jj=1:length(nodes)})
    end
end

rules = Dict{ASCIIString,Set{State{Int}}}()
for class_index in 1:length(unordered_markers)
    w_class =  unordered_markers[class_index]
    if w_class∉(START_MARKER1,START_MARKER2)
        #Not rquired to make a transition so that START_MARKER1 or 2 ever occur in second position
        jjs = node_indexes_for_2nd[class_index]
        @addConstraint(m, sum{x[ii,jj],ii=1:length(nodes), jj=jjs}==1)
        rules["*"*w_class]=Set(product(1:length(nodes),jjs))
    end
    
    if w_class∉(END_MARKER1,END_MARKER2)
        #Not rquired to make a transition so that END_MARKER1 or 2 ever occur in second position
        iis=node_indexes_for_1st[class_index]
        @addConstraint(m, sum{x[ii,jj], ii=iis,jj=1:length(nodes)}==1)
        rules[w_class*"*"]=Set(product(iis,1:length(nodes)))
    end
end

banned_trans_prob = Set{State{Int}}()
banned_trans_state = Set{State{Int}}()
log_trans_prob = spzeros(length(nodes), length(nodes))
for (from_node_index, from_node) in enumerate(nodes)
    w1 = unordered_markers[from_node[1]]
    w2 = unordered_markers[from_node[2]]
    #If what was in the second state element does not end up in the first state element then it is not allowed.
    can_transition_to = Set(get(node_indexes_for_1st, from_node[2], Int[]))
        #You can transition to any node which has your second element as its first element
    for (to_node_index, to_node) in enumerate(nodes)
        if to_node_index in can_transition_to
            @assert(from_node[2]==to_node[1])
            w3= unordered_markers[to_node[2]]
            log_tp = log_lm(w1,w2,w3)
            if log_tp>-Inf
                log_trans_prob[from_node_index, to_node_index] = log_tp
                continue
            else
                #Banned as prob zero transitions not allowed
                @addConstraint(m, x[from_node_index,to_node_index]==0)
                push!(banned_trans_prob, (from_node_index,to_node_index))
            end
            
        else
            #It is not a legal transition
            @addConstraint(m, x[from_node_index,to_node_index]==0)
            push!(banned_trans_state, (from_node_index,to_node_index))
        end
    end
end



function eleminate_subtours(cb)
    #@printval MathProgBase.cbgetstate(cb)
    #@printval cb
    subtours = get_subtours(x)
    if length(subtours)==1
        #no subtour to eleminate
        return
    end
    subtour_hyperclasses = @pipe subtours |> map(get_hyperclass,_)
    
    
    #Unless this is the First subtour's hyperclass, then there must be away into this hyperclass from another (outside in general)
    for subtour_hyperclass in subtour_hyperclasses[2:end]
        arcs_into_subtour = AffExpr()
        for jj in subtour_hyperclass
            for ii in 1:length(nodes)
                if ii∉subtour_hyperclass
                    arcs_into_subtour += x[ii,jj]
                end
            end
        end
        @addLazyConstraint(cb, arcs_into_subtour >= 1)
    end
    
    #Also the First subtour must go to one of the other subtours 
    arcs_outof_subtour = AffExpr()
    for ii in subtour_hyperclasses[1]
        for jj in 1:length(nodes)
            if jj∉subtour_hyperclasses[1]
                arcs_outof_subtour += x[ii,jj]
            end
        end
    end
    @addLazyConstraint(cb, arcs_outof_subtour >=1)

end

addLazyCallback(m, eleminate_subtours,fractional=false)

#new Linear way using logprobs
@setObjective(m, Max, sum{log_trans_prob[i,j]*x[i,j], i=1:length(nodes), j=1:length(nodes)})

toc()
@printval length(unordered_markers)
@printval length(m.linconstr)
log_trans_prob


elapsed time: 2

228x228 sparse matrix with 768 Float64 entries:
	[33 ,   2]  =  -1.38629
	[89 ,   2]  =  -1.38629
	[103,   2]  =  -1.51989e-5
	[117,   2]  =  -0.0870114
	[215,   2]  =  -0.133531
	[1  ,   4]  =  -3.48924
	[1  ,   7]  =  -5.9053
	[1  ,   8]  =  -3.92215
	[1  ,   9]  =  -6.98176
	[1  ,  10]  =  -2.12384
	⋮
	[158, 227]  =  -8.81159
	[186, 227]  =  -8.15133
	[17 , 228]  =  -8.41892
	[46 , 228]  =  -10.5722
	[74 , 228]  =  -10.0901
	[88 , 228]  =  -10.421
	[102, 228]  =  -10.09
	[130, 228]  =  -10.0902
	[144, 228]  =  -10.0901
	[158, 228]  =  -10.421
	[186, 228]  =  -10.4885

In [124]:
@time solve(m)
println("Objective value: ", getObjectiveValue(m))
println("Prob: ", e^getObjectiveValue(m))
x_val = getValue(x)
#println("x = ", x_val)
x_iis,x_jjs, _  = findnz(x_val.>1-1e-6)
zip(x_iis,x_jjs) |> collect |> println

.596589957 seconds
length(unordered_markers) = 18
length(m.linconstr) = 51474
1 11
11 138
138 128
128 191
191 84
84 168
168 150
150 99
99 176
176 74
74 228
228 202
202 34
34 52
52 103
103 2
1 11
11 135
135 88
88 228
228 202
202 34
34 52
52 103
103 2
100 198
198 176
176 67
67 126
126 168
168 150
150 100
1 17
17 228
228 202
202 34
34 52
52 103
103 2
96 141
141 176
176 67
67 128
128 191
191 84
84 168
168 150
150 96
1 17
17 228
228 205
205 79
79 96
96 141
141 176
176 67
67 128
128 196
196 154
154 160
160 34
34 52
52 103
103 2
1 8
8 100
100 191
191 81
81 124
124 141
141 176
176 74
74 228
228 202
202 34
34 52
52 103
103 2
168 154
154 168
1 8
8 100
100 196
196 152
152 124
124 141
141 176
176 74
74 228
228 202
202 34
34 52
52 103
103 2
84 163
163 84
1 11
11 139
139 158
158 228
228 202
202 34
34 52
52 103
103 2
100 198
198 176
176 67
67 126
126 163
163 79
79 100
1 11
11 139
139 152
152 126
126 163
163 79
79 100
100 198
198 176
176 74
74 228
228 202
202 34
34 52
52 103
103 2
128.122866 seconds (

In [127]:
nodes_chain = Dict([ii=>jj for (ii,jj) in zip(x_iis,x_jjs)])
node_index=1
visitted = Set{Int}()
while(node_index!=2)
    push!(visitted, node_index)
    println(unordered_markers[nodes[node_index][1]])
    #println(node_index, " ", unordered_markers[nodes[node_index][1]]," ", unordered_markers[nodes[node_index][2]])
    node_index = nodes_chain[node_index]
end

**START1**
**START2**
first
of
the
manners
of
this
basis
is
performed
a
comedy
in
1892
.


In [106]:
x_val = getValue(x)
x_iis,x_jjs, _  = findnz(x_val)
nodes_chain = Dict([ii=>jj for (ii,jj) in zip(x_iis,x_jjs)])

Dict{Any,Any} with 12 entries:
  68  => 54
  54  => 119
  77  => 35
  62  => 103
  35  => 2
  10  => 86
  29  => 68
  119 => 62
  103 => 111
  111 => 77
  86  => 29
  1   => 10

**START2**
this
is
the
basis
of
a
comedy
of
manners
.


In [ ]:
x

In [ ]:
collect(1:100)[[1]]

In [ ]:
sts = get_subtours(x)

In [ ]:
1+1

In [ ]:
for tran in union(banned_trans_prob, banned_trans_state)
    for rule in values(rules)
        delete!(rule, tran)
    end
end
rules

In [ ]:
rules["fine*"] ∩ keys(nodes_chain)

In [ ]:
function get_prod(x, iis, jjs)
    net = 1.0
    for i in 1:size(x,1) 
        for j in 1:size(x,2)
            println(i,",", j, " =", x[i,j], " ", trans_prob[i,j])
            net*=max((x[i,j]-1)^2, trans_prob[i,j])
        end
    end
    net
end
values = falses(size(x))
values[1,3] = 1
values[3,6] = 1
values[6,7] = 1
println("----------\n")
get_prod(values, iis,jjs)
